In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from pycaret.regression import *

from ETL import ETL

plt.style.use("seaborn-v0_8")

In [2]:
df_yield = pd.read_csv("data/barley_yield_from_1982.csv", sep=";")
df_climate = pd.read_parquet("data/climate_data_from_1982.parquet")

In [3]:
df_yield, df_climate = ETL(df_yield, df_climate).run()

Departments/Scenario dropped because of missing values:                           nom_dep  scenario
date                                       
2015-01-01               Calvados  ssp2_4_5
2015-01-01            Deux_Sevres  ssp2_4_5
2015-01-01                Essonne  ssp2_4_5
2015-01-01                   Eure  ssp2_4_5
2015-01-01                  Rhone  ssp2_4_5
2015-01-01        Tarn_et_Garonne  ssp2_4_5
2015-01-01  Territoire_de_Belfort  ssp2_4_5
2015-01-01               Vaucluse  ssp2_4_5
Departments dropped because of almost absolute absence of data:
 ['Hauts_de_Seine' 'Paris' 'Seine_SeineOise']
